In [1]:
import pandas as pd
import json
import numpy as np
# prod_arg = ['category', 'description', 'title', 'brand', 'main_cat', 'price', 'asin', 'imageURL', 'imageURLHighRes']
# rat_arg=['overall', 'reviewerID','asin', 'reviewerName']
rating_df = pd.read_csv('Home_and_Kitchen.csv', usecols = ['0006564224', 'A3NSN9WOX8470M', '5.0'])
rating_df.rename(columns={"0006564224":"asin","A3NSN9WOX8470M":"reviewerID","5.0":"overall"}, inplace=True)
# rating_df = rating_df[['overall', 'reviewerID','asin', 'reviewerName']]
products_df = pd.read_json('meta_Home_and_Kitchen.json', lines=True, nrows=150000)
products_df = products_df[['category', 'description', 'title', 'brand', 'main_cat', 'price', 'asin', 'imageURL', 'imageURLHighRes']]

In [2]:
print(rating_df.shape)
print(products_df.shape)
rating_df.tail(5)

(21928567, 3)
(150000, 9)


,asin,reviewerID,overall
21928562,B01HJH7K3Q,A33Z5MOHLFIECI,3.0
21928563,B01HJH7K3Q,A1BP93Y9L2HUL7,5.0
21928564,B01HJH7K3Q,A3EYW1FBJ48SH1,5.0
21928565,B01HJHTC6O,AYOGJY5CDIY49,5.0
21928566,B01HJHTC6O,ADNZ2NWXVKB1L,5.0


In [5]:
rating_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21928567 entries, 0 to 21928566
Data columns (total 3 columns):
 #   Column      Dtype  
---  ------      -----  
 0   asin        object 
 1   reviewerID  object 
 2   overall     float64
dtypes: float64(1), object(2)
memory usage: 501.9+ MB


# Popularity based recommendations

In [12]:
rat_count_df=rating_df.groupby('asin').count()['overall'].reset_index()
rat_count_df.rename(columns={"overall":"rating-count"}, inplace=True)
rat_count_df

,asin,rating-count
0,0001487795,6
1,0002020300,1
2,0006564224,11
3,0009046461,1
4,0234937912,1
...,...,...
1286045,B01HJGJUF8,30
1286046,B01HJH0J4S,2
1286047,B01HJH7K3Q,3
1286048,B01HJHOITU,2


In [4]:
rat_avg_df = rating_df.groupby('asin').mean()['overall'].reset_index()
rat_avg_df.rename(columns={"overall":"rating-avg"}, inplace=True)
rat_avg_df


C:\Users\surya\AppData\Local\Temp\ipykernel_12704\2900839084.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  rat_avg_df = rating_df.groupby('asin').mean()['overall'].reset_index()


,asin,rating-avg
0,0001487795,5.000000
1,0002020300,1.000000
2,0006564224,3.818182
3,0009046461,2.000000
4,0234937912,5.000000
...,...,...
1286045,B01HJGJUF8,4.566667
1286046,B01HJH0J4S,4.000000
1286047,B01HJH7K3Q,4.333333
1286048,B01HJHOITU,5.000000


In [13]:
popular_df = rat_avg_df.merge(rat_count_df,on="asin")
popular_df = popular_df[popular_df['rating-count'] > 500].sort_values('rating-count', ascending=False).reset_index()
popular_df

,index,asin,rating-avg,rating-count
0,610510,B00FLYWNYQ,4.590940,27595
1,432611,B009HVH4XO,4.280369,15822
2,75578,B000YGEVMI,4.341422,15784
3,874684,B00Q7EV29G,4.164734,15698
4,579432,B00EINBSJ2,4.343651,15443
...,...,...,...,...
6309,406070,B008O5DG3Q,4.746507,501
6310,932178,B00U86XNM8,4.752495,501
6311,102915,B001B1C4G0,4.532934,501
6312,868576,B00PTRD3DU,3.810379,501


In [33]:
best_products = products_df.merge(popular_df,on="asin").sort_values('rating-count', ascending=False).reset_index()
# display(best_products.head(10))
print(best_products.shape)
best_products['title'].drop_duplicates(inplace=True)
best_products

(2113, 13)


,level_0,category,description,title,brand,main_cat,price,asin,imageURL,imageURLHighRes,index,rating-avg,rating-count
0,1508,"[Home & Kitchen, Kitchen & Dining, Travel & To...",[THERMOS products deliver what matters every t...,"Thermos FUNtainer 12 Ounce Bottle, #All About Me",Thermos,Amazon Home,$16.93,B000YGEVMI,[],[],75578,4.341422,15784
1,1663,"[Home & Kitchen, Kitchen & Dining, Coffee, Tea...",[Made from luxurious stainless steel. Makes 4 ...,Presto 02811 12-Cup Stainless Steel Coffee Maker,Presto,Amazon Home,$6.52,B0014CX87U,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,85801,4.295427,11414
2,1596,"[Home & Kitchen, Kitchen & Dining, Kitchen Ute...","[American Weigh Digital Scale, 100g [0.01g sen...",American Weigh Scale AWS-100 Digital Pocket Sc...,American Weigh Scales,Amazon Home,$10.47,B0012LOQUQ,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,81367,4.155495,10682
3,1708,"[Home & Kitchen, Kitchen & Dining, Small Appli...",[The Original Beater Blade by New Metro Design...,Original Beater Blade for 5-Quart KitchenAid B...,New Metro Design,Amazon Home,$21.22,B0015TMHSI,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,88875,3.967221,9915
4,1831,"[Home & Kitchen, Home Dcor, Candles & Holders,...","[, Comes with 15-watt bulb, Dimmer switch and ...","Himalayan Glow 1002 Pink Crystal Salt Lamp, (8...",Himalayan Glow,Tools & Home Improvement,$19.99,B001892AX2,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,95371,4.102756,9216
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2108,577,"[Home & Kitchen, Kitchen & Dining, Bakeware, D...","[Ateco 1447 Decorating Comb & Icing Smoother, ...",Ateco 1447 Decorating Comb &amp; Icing Smoothe...,Ateco,Amazon Home,$1.71,B0000VOOXI,[],[],9923,4.518962,501
2109,235,"[Home & Kitchen, Kitchen & Dining, Dining & En...","[Anchor Hocking Presence Cake Plate w/Dome, 2 ...","Anchor Hocking Presence Cake Plate w/Dome, 2 P...",Anchor Hocking,Amazon Home,,B0000630NY,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,3945,4.265469,501
2110,236,"[Home & Kitchen, Kitchen & Dining, Dining & En...","[Anchor Hocking Presence Cake Plate w/Dome, 2 ...","Anchor Hocking Presence Cake Plate w/Dome, 2 P...",Anchor Hocking,Amazon Home,,B0000630NY,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,3945,4.265469,501
2111,707,"[Home & Kitchen, Kitchen & Dining, Storage & O...",[Support USA Companies - Buy from USA - Bread ...,Buddeez Sandwich Size Bread Buddy Dispenser,Buddeez,Amazon Home,$11.38,B00024JRC6,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,13063,4.399202,501


# ITEM - ITEM collaborative filtering

In [15]:
temp_y = rating_df.groupby('reviewerID').count()['overall'] > 30
customerbyvote = temp_y[temp_y].index
customerbyvote

Index(['A00577669P2XCLU956L8', 'A0490453AYGB1BAAOG64', 'A0718784TOWF9DX1DOFD',
       'A1007QP6OB28XH', 'A100ELBI8BSXR1', 'A100L918633LUO', 'A100UD67AHFODS',
       'A100WO06OQR8BQ', 'A1017UZIPW58F4', 'A101ONEL0VSYAV',
       ...
       'AZWCQL30IJZR8', 'AZWQ56HWVYO0Q', 'AZX2RDN9YXZAE', 'AZXFS8GCTSQ5R',
       'AZXP46IB63PU8', 'AZYP4FQ2L2C4O', 'AZZRRYBQG57LF', 'AZZT1ERHBSNQ8',
       'AZZV9PDNMCOZW', 'AZZXCFBNEWIBQ'],
      dtype='object', name='reviewerID', length=17150)

In [16]:
filtered_user = rating_df[rating_df['reviewerID'].isin(customerbyvote)]
filtered_user

,asin,reviewerID,overall
18,0681795107,A2RQ7VLAK1SHPU,1.0
61,0768205921,A1KU2VHK8BECRW,5.0
63,0768205921,A2114M46F4MO2W,5.0
125,1574893122,AK3UOLPIJ7FH8,5.0
197,1605161810,AG6BH7V520TJ5,5.0
...,...,...,...
21928272,B01HJ8BKEK,A24T2P7NW63GOG,4.0
21928303,B01HJ8C1N4,A1FP4HXOVMQHOF,1.0
21928416,B01HJATSR4,A25J9X5ZRUPT8K,4.0
21928464,B01HJEM0IY,A2AO95W94NTAOT,4.0


In [17]:
temp_x = filtered_user.groupby('asin').count()['overall'] > 50
products_gt_50 = temp_x[temp_x].index
products_gt_50

Index(['B00002N602', 'B00002N8CX', 'B00004OCIP', 'B00004OCJN', 'B00004OCKR',
       'B00004OCL2', 'B00004OCL8', 'B00004OCLX', 'B00004OCM3', 'B00004OCMD',
       ...
       'B01FSC4AGE', 'B01G5LGEUC', 'B01G7SBOF8', 'B01G7SGOL2', 'B01G9DDFNA',
       'B01GBT5I0A', 'B01GD4VHEE', 'B01GJ0XT0C', 'B01HBC1K1W', 'B01HC7BNJA'],
      dtype='object', name='asin', length=1415)

In [18]:
filtered_product = filtered_user[filtered_user['asin'].isin(products_gt_50)]
filtered_product = filtered_product.merge(products_df, on='asin')[['asin', 'overall', 'reviewerID', 'title', 'price', 'category', 'imageURLHighRes']]
filtered_product

,asin,overall,reviewerID,title,price,category,imageURLHighRes
0,B00002N602,2.0,A2UGT81M7AU4WA,Presto 01341 4-Quart Stainless Steel Pressure...,$5.54,"[Home & Kitchen, Kitchen & Dining, Cookware, P...",[https://images-na.ssl-images-amazon.com/image...
1,B00002N602,2.0,A2UGT81M7AU4WA,Presto 01341 4-Quart Stainless Steel Pressure...,$5.54,"[Home & Kitchen, Kitchen & Dining, Cookware, P...",[https://images-na.ssl-images-amazon.com/image...
2,B00002N602,5.0,AACANL2B46MPB,Presto 01341 4-Quart Stainless Steel Pressure...,$5.54,"[Home & Kitchen, Kitchen & Dining, Cookware, P...",[https://images-na.ssl-images-amazon.com/image...
3,B00002N602,5.0,AACANL2B46MPB,Presto 01341 4-Quart Stainless Steel Pressure...,$5.54,"[Home & Kitchen, Kitchen & Dining, Cookware, P...",[https://images-na.ssl-images-amazon.com/image...
4,B00002N602,3.0,A3OEGVSXY0Y07W,Presto 01341 4-Quart Stainless Steel Pressure...,$5.54,"[Home & Kitchen, Kitchen & Dining, Cookware, P...",[https://images-na.ssl-images-amazon.com/image...
...,...,...,...,...,...,...,...
72031,B000EE5DTK,3.0,A3OS1SO09S3WRJ,Water Pillow by Mediflow: The first and origin...,$44.99,"[Home & Kitchen, Bedding, Bed Pillows & Positi...",[https://images-na.ssl-images-amazon.com/image...
72032,B000EE5DTK,5.0,AG8RT4H7GY6EB,Water Pillow by Mediflow: The first and origin...,$44.99,"[Home & Kitchen, Bedding, Bed Pillows & Positi...",[https://images-na.ssl-images-amazon.com/image...
72033,B000EE5DTK,5.0,A1AJ8EDOWYY1SP,Water Pillow by Mediflow: The first and origin...,$44.99,"[Home & Kitchen, Bedding, Bed Pillows & Positi...",[https://images-na.ssl-images-amazon.com/image...
72034,B000EE5DTK,5.0,A3S3R88HA0HZG3,Water Pillow by Mediflow: The first and origin...,$44.99,"[Home & Kitchen, Bedding, Bed Pillows & Positi...",[https://images-na.ssl-images-amazon.com/image...


In [19]:
pivottable = filtered_product.pivot_table(index='asin', columns='reviewerID', values='overall')
pivottable.fillna(0,inplace=True)
pivottable

reviewerID,A0490453AYGB1BAAOG64,A0718784TOWF9DX1DOFD,A100L918633LUO,A100UD67AHFODS,A100WO06OQR8BQ,A1017UZIPW58F4,A101ONEL0VSYAV,A1027WM3PXKVMR,A102UXGLDF76G1,A103G18P12I5WL,...,AZVK1493WP0GL,AZWAEIDGU5JJB,AZWCQL30IJZR8,AZWQ56HWVYO0Q,AZX2RDN9YXZAE,AZXP46IB63PU8,AZZRRYBQG57LF,AZZT1ERHBSNQ8,AZZV9PDNMCOZW,AZZXCFBNEWIBQ
asin,,,,,,,,,,,,,,,,,,,,,
B00002N602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00002N8CX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004OCIP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004OCJN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00004OCKR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B001P5LMTM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B001P5YUM8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B001PB8EJ2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_dist=cosine_similarity(pivottable)
cosine_dist

array([[1.        , 0.        , 0.00940028, ..., 0.        , 0.        ,
        0.01003703],
       [0.        , 1.        , 0.01540526, ..., 0.05093498, 0.        ,
        0.00913821],
       [0.00940028, 0.01540526, 1.        , ..., 0.02697212, 0.03918135,
        0.01417151],
       ...,
       [0.        , 0.05093498, 0.02697212, ..., 1.        , 0.01077162,
        0.00857118],
       [0.        , 0.        , 0.03918135, ..., 0.01077162, 1.        ,
        0.        ],
       [0.01003703, 0.00913821, 0.01417151, ..., 0.00857118, 0.        ,
        1.        ]])

In [34]:
def recommender(item_name):
    index = np.where(pivottable.index==item_name)[0][0]
    similar = list(enumerate(cosine_dist[index]))
    similar.sort(key=lambda a:a[1], reverse=True)
    data=[]
    for i in similar:
        item=[]
        temp_df=popular_df[best_products['asin']==pivottable.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('title')['asin'].values))
        item.extend(list(temp_df.drop_duplicates('title')['rating-avg'].values))
        item.extend(list(temp_df.drop_duplicates('title')['rating-count'].values))
        data.append(item)
#         k=pivottable.index[i[0]]
#         print('*'*100)
#         print(k)
#         print(best_products.loc[best_products['asin'] == k]['title'].to_string())

    return data

In [35]:
item = "B00004OCKR"
print(best_products.loc[best_products['asin'] == item]['title'].to_string())
print("=============================")
recommender(item)

66    OXO Good Grips Salad Spinner, Medium, Clear
67    OXO Good Grips Salad Spinner, Medium, Clear


C:\Users\surya\AppData\Local\Temp\ipykernel_12704\711949425.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_df=popular_df[best_products['asin']==pivottable.index[i[0]]]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [23]:
popular_df

,index,asin,rating-avg,rating-count
0,610510,B00FLYWNYQ,4.590940,27595
1,432611,B009HVH4XO,4.280369,15822
2,75578,B000YGEVMI,4.341422,15784
3,874684,B00Q7EV29G,4.164734,15698
4,579432,B00EINBSJ2,4.343651,15443
...,...,...,...,...
6309,406070,B008O5DG3Q,4.746507,501
6310,932178,B00U86XNM8,4.752495,501
6311,102915,B001B1C4G0,4.532934,501
6312,868576,B00PTRD3DU,3.810379,501


In [36]:
import pickle
pickle.dump(popular_df, open('popular_df_hk.pkl','wb'))

In [37]:
import pickle
pickle.dump(best_products, open('best_products_hk.pkl','wb'))

In [39]:
import pickle
pickle.dump(cosine_dist, open('cosine_dist_hk.pkl','wb'))